# Kalibrace kamery a odstranění soudkovitosti
Ze začátku notebooku jsou nejdříve pomocné funkce. [Úkol](#Úkol) následuje níže.

### Import knihoven a konfigurace

In [1]:
import argparse
import os
import shutil
import re

import numpy as np
import matplotlib.pyplot as plt
import cv2
import yaml

from natsort import natsorted

from pypylon import pylon 
from pypylon_opencv_viewer import BaslerOpenCVViewer

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

<a id='load_save_functions'>Funkce sloužící k načítaní/ukládání kalibrace ze souboru.</a>

In [2]:
IDX_CAM_MATRIX = "camera_matrix"
IDX_DIST_COEFFS = "dist_coefs"

def load_camera_calib(input_file):
    """ Loads camera calibration from specified input file.
    
    Parameters
    ----------
    input_file : string
        Input file with calibration data in YAML format.
    Returns
    -------
    tuple(np.array, np.array)
        Returns a tuple where first element is camera matrix array and second element is dist coefficients array. 
        These arrays might be empty if the file isn't found or in correct format.
    """
    try:
        with open(input_file, 'r') as stream:
            data = yaml.load(stream)
            return data[IDX_CAM_MATRIX], data[IDX_DIST_COEFFS]
    except (FileNotFoundError, yaml.YAMLError) as exc:
        print(f'File {input_file} couldn\'t be read.')
        return np.array([]), np.array([])


def save_camera_calib(output_file, camera_matrix, dist_coefs):
    """ Saves camera calibration to specified output file.
    
    Parameters
    ----------
    output_file : string
        Output file used for storing calibration data in YAML format. Parent directory is created if needed.
    Returns
    -------
    None
    """
    data = {IDX_CAM_MATRIX: camera_matrix, IDX_DIST_COEFFS: dist_coefs}
    output_dir = os.path.dirname(output_file)
    
    if not os.path.isdir(output_dir):
            os.mkdir(output_dir)
            
    with open(output_file, "w") as f:
        yaml.dump(data, f)

<a id='correct_functions'>Funkce k opravě obrázku pomocí získaných kalibračních dat.</a>

In [3]:
def correct_frame(frame, camera_matrix, dist_coeffs):
    """Returns undistorted frame."""
    return cv2.undistort(frame, camera_matrix, dist_coeffs)

<a id='connect_functions'>Funkce k připojení kamery podle sériového čísla.</a> Umožňuje poté tlačítkem zapnout snímání aktuální scény. V případě nutnosti změny nastavení připojení kamery, je třeba restartovat jupyter notebook tlačítkem `restart kernel (with dialog)`. Kamera se bude tvářit jako, že je již připojená k jinému zařízení. (`RuntimeException: Failed to open 'Basler ...'. The device is controlled by another application.`)

In [4]:
CAM_WIDTH = 1080
CAM_HEIGHT = 720

def connect_camera(serial_number, grabbed_images_path):
    ''' Connects camera specified with its serial number
    
    Parameters
    ----------
    serial_number : string
        Camera's serial number.
    grabbed_images_path : string
        Path to folder where the saved images will be stored.
    Returns
    -------
    None
    '''
    info = None
    for i in pylon.TlFactory.GetInstance().EnumerateDevices():
        if i.GetSerialNumber() == serial_number:
            info = i
            break
    else:
        print('Camera with {} serial number not found'.format(serial_number))

    # VERY IMPORTANT STEP! To use Basler PyPylon OpenCV viewer you have to call .Open() method on you camera
    if info is not None:
        camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateDevice(info)) 
        camera.Open()
            
    viewer = BaslerOpenCVViewer(camera)
    viewer.run_interaction_continuous_shot(window_size=(CAM_WIDTH, CAM_HEIGHT), image_folder=grabbed_images_path)

<a id='show_functions'>Funkce k zobrazení obecného množství obrázků ze vstupu.</a>

In [5]:
def show_images(*imgs, window_name='Image preview'):
    """ Opens multiple image previews depending on the length of the input *imgs list.
    The preview is terminated by pressing the 'q' key.
    
    Parameters
    ----------
    *imgs : list
        Multiple input images which have to be shown.
    window_name : Optional[string]
        An optional window name.
    Returns
    -------
    None
    """
    for i, img in enumerate(imgs, 1):
        window_name_id = window_name + ' ' + str(i)
        cv2.namedWindow(window_name_id, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
        cv2.resizeWindow(window_name_id, 1080, 720)
        cv2.moveWindow(window_name_id, 0, 0)

    while 1:
        for i, img in enumerate(imgs, 1):
            cv2.imshow(window_name + ' ' + str(i), img)
            
        k = cv2.waitKey(1) & 0xFF

        if k == ord('q'):
            break

    cv2.destroyAllWindows()

<a id='reindex_functions'>Funkce sloužící k přejmenování souborů ve složce</a>, aby je bylo možné snadné použít v cv2.VideoCapture.

In [6]:
def reindex_image_files(source_dir, output_dir=None):
    """ Reads all images in source_dir and based on they original order, 
    change their filename to be continuous integer (starting from 0). 
    Then, they can be easily read by cv2.VideoCapture. Image format is kept.
    
    Parameters
    ----------
    source_dir : string
        Input images directory that have to be renamed.
    output_dir : Optional[string]
        Output directory for renamed files. If not specified, renaming is done inplace in source_dir.
    Returns
    -------
    None
    """
    input_files = []
    
    for file in os.listdir(source_dir):
        if re.match(r'.*(\.bmp|\.jpg|\.png|\.gif)$', file, re.I):
            input_files.append(os.path.join(source_dir, file))

    if not input_files:
        print('No files were found.')
        return
    
    extension = '.' + input_files[0].split(".")[-1]
    if output_dir is None:
        for i, filename in enumerate(natsorted(input_files)):
            os.rename(filename, os.path.join(source_dir, str(i) +  extension))
        print(f'Files within {source_dir} were renamed, starting from 0{extension} to {i}{extension}.')
    else:
        if not os.path.isdir(output_dir):
            os.mkdir(output_dir)

        for i, filename in enumerate(natsorted(input_files)):
            shutil.copy(filename, os.path.join(output_dir, str(i) + extension))
            
        print(f'Files from {source_dir} were renamed and saved to {output_dir}, starting from 0{extension} to {i}{extension}.')

<a id='pick_functions'>Funkce k rychlému výběru chtěných snímků z obrazového zdroje.</a>

In [7]:
def pick_frames(input_source, output_dir, wait_time):
    """ Sequentially shows all images from input_source. 
    Using 'p' key allows to pick wanted frames and save them in separate folder (output_dir).
    The preview is terminated by pressing the 'q' key.
    
    Parameters
    ----------
    input_source : string
        Input source for cv2.VideoCapture (could be camera source or sequence of saved images where format has to be specified).
    output_dir : string
        Output directory for picked files. Automatically created if needed.
    wait_time : int
        Delay in ms between shown images.
    Returns
    -------
    None
    """
    window_name = 'Frame preview'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, CAM_WIDTH, CAM_HEIGHT)
    cv2.moveWindow(window_name, 0, 0)
    
    cap = cv2.VideoCapture(input_source)
    
    if not cap.isOpened():
        cv2.destroyAllWindows()
        raise FileNotFoundError('Capture cannot be opened.')
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    
    saved_counter = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: 
            break
            
        cv2.imshow(window_name, frame)
        k = cv2.waitKey(wait_time) & 0xFF
    
        if k == ord('p'):
            img_format = input_source.split('.')[-1] if '.' in input_source else 'jpg'
            cv2.imwrite(os.path.join(output_dir, str(saved_counter) + '.' + img_format), frame)
            saved_counter += 1
        elif k == ord('q'):
            break

    cv2.destroyAllWindows()
    print(f'{saved_counter} frames saved to {output_dir}')          

<a id='calib_functions'>Funkce provádějící kalibraci kamery</a>, kterou je možné uložit k pozdějšímu využití.

In [8]:
def camera_calib(input_source, chess_shape, output_calib_file=None, img_show_delay=1):
    """ Browses all images found in input_source and on each image tries to find chessboard corners.
    If chessboard corners are found, image corespondences with real world space are added to lists.
    Based on these image-world corespondences, camera calibration is made.
    
    Parameters
    ----------
    input_source : string
        Input source for cv2.VideoCapture (could be camera source or sequence of saved images where format has to be specified).
    chess_shape : tuple
        Number of inner corners per a chessboard row and column.
    output_calib_file : Optional[string]
        Output file where calibration is saved when neccesary.
    img_show_delay : int
        Delay in ms between shown images.
    Returns
    -------
    tuple
        camera matrix and distance coefficients
    """
    window_name = 'Frame preview'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, CAM_WIDTH, CAM_HEIGHT)
    cv2.moveWindow(window_name, 0, 0)

    cap = cv2.VideoCapture(input_source)

    if not cap.isOpened():
        cv2.destroyAllWindows()
        raise FileNotFoundError('Capture cannot be opened.')

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((chess_shape[0] * chess_shape[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:chess_shape[0], 0:chess_shape[1]].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real world space
    imgpoints = []  # 2d points in image plane.

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, chess_shape)

        # If found, add object points, image points (after refining them)
        if ret:
            objpoints.append(objp)
            corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1))
            imgpoints.append(corners)

            # Draw and display the corners
            frame = cv2.drawChessboardCorners(frame, chess_shape, corners, ret)

        cv2.imshow(window_name, frame)

        k = cv2.waitKey(img_show_delay) & 0xFF
        if k == ord('q'):
            break

    cv2.destroyAllWindows()
    
    print('Computing camera matrix...')
    rms, camera_matrix, dist_coefs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    if output_calib_file is not None:
        save_camera_calib(output_calib_file, camera_matrix, dist_coefs)

    print('\nRMS:', rms)
    print('Camera matrix:\n', camera_matrix)
    print('Distortion coefficients: ', dist_coefs.ravel())
    
    return camera_matrix, dist_coefs

<a id='path_folder_functions'>Funkce pro vytvoření cesty z názvů složek.</a>

In [9]:
def create_folder_path(base_folder, new_folder_name):
    """ Creates all neccessary folders in the folder tree structure on computer. 
    
    Parameters
    ----------
    base_folder : string
        Base folder directory in string notation. 
    output_dir : string
        Folder name that should be inside the base folder.
    Returns
    -------
    string
        Path to the newly created folder.
    """
    if not os.path.isdir(base_folder):
        os.mkdir(base_folder)
            
    path = os.path.join(base_folder, new_folder_name)        
    
    if not os.path.isdir(path):
        os.mkdir(path)      
            
    return path

<a id='path_file_functions'>Funkce pro vytvoření cesty z názvu složky a názvu souboru.</a>

In [10]:
def create_file_path(folder, file_name):
    '''Easier defined function to create path for filename inside a folder.
    
    Parameters
    ----------
    folder : string
        Base folder directory in string notation. 
    file_name : string
        File name that should be inside the base folder.
    Returns
    -------
    string
        Path to the newly created file.
    """
    '''
    if not os.path.isdir(folder):
        os.mkdir(folder)
        
    return os.path.join(folder, file_name)

---

Seznam funkcí pro přehlednost:
- [`load_camera_calib(...)`](#load_save_functions)
- [`save_camera_calib(...)`](#load_save_functions)
- [`correct_frame(...)`](#correct_functions)
- [`connect_camera(...)`](#connect_functions)
- [`show_images(...)`](#show_functions)
- [`reindex_image_files(...)`](#reindex_functions)
- [`pick_frames(...)`](#pick_functions)
- [`camera_calib(...)`](#calib_functions)
- [`create_folder_path(...)`](#path_folder_functions)
- [`create_file_path(...)`](#path_file_functions)

### Úkol
Úkol je zaměřen na kalibraci kamery a následné odstranění soudkovitosti ze snímků. K tomu, aby bylo možné provést kalibraci, je zapotřebí dostatečný počet snímků (10-30), které obsahují předem známý vzor (v našem případě šachovnice), zaznamenaný v různých úhlech. Knihovna OpenCV následně sama tyto vzory vyhledá, vypočte kamerovou matici (camera matrix) a parametry zkreslení (distortion coefficients). 
Kamera musí být při snímání stacionárně upevněna, nejlépe na stativu/stojanu.

Dobrovolné:
Více na téma kalibrace se lze dočíst [zde](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html) a [zde](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html).

#### 1) Získejte záznam snímků s šachovnicí ke kalibraci pomocí funkce pro zobrazení snímků z Basler kamery. Správně zvolte sériové číslo kamery a složku, ve které se bude pracovat.

In [ ]:
serial_number = ... ###
base_folder = ... ###
grabbed_images_folder_path = ...(base_folder, 'grabbed_images') ###

...(serial_number, grabbed_images_folder_path) ###

#### 2) Zavolejte funkci pro přejmenování souborů tak, aby šly snadno přečíst pomocí cv2.VideoCapture.

In [ ]:
...(grabbed_images_folder_path) ###

#### 3) Zavolejte funkci pro výběr snímků. Zvolte ty, které budou použity ke kalibraci. Zvolte si složku pro vybrané snímky.

In [ ]:
images_format = '%01d.png'
picked_images_folder_path = create_folder_path(grabbed_images_folder_path, 'picked')
wait_time = ... ### v ms

...(create_file_path(grabbed_images_folder_path, images_format), picked_images_folder_path, wait_time=wait_time) ###

#### 4) V případě, že budete ručně zasahovat do vybraných snímků (např. mazáním), proveďte znovu přejmenování.

In [ ]:
...(picked_images_folder_path) ###

#### 5) Zvolte název souboru pro uložení kalibrace kamery. Prověďte kalibraci kamery. Nezapomeňte zvolit správnou velikost šachovnice.

In [ ]:
calibration_file_name = ... ### *.yaml
output_calib_file_path = create_file_path(base_folder, calibration_file_name)
chess_shape= ... ###
camera_matrix, dist_coefs = ...(create_file_path(picked_images_folder_path, images_format), chess_shape=chess_shape, output_calib_file=output_calib_file_path) ###

#### 6) Načtěte zdrojový obrázek. Ten opravte pomocí funkce na odstranění soudkovitosti. Oba obrázky zobrazte.

In [ ]:
img_raw = ...(create_file_path(picked_images_folder_path, '0.png')) ###
img_corrected = ...(img_raw, camera_matrix, dist_coefs) ###

...(img_raw, img_corrected) ###